# 3rd May April: Morphine + Covid

In [13]:
import json
import torch
import os
import pandas as pd
import numpy as np
from pathlib import Path
from tqdm import tqdm
import random
import openai

from pathlib import Path
from openai import OpenAI
from agatha.construct.semrep_handler import SemRepHandler
from subgraph_retrieval import GraphContextManager
from sentence_transformers import SentenceTransformer, util
import warnings
warnings.filterwarnings('ignore')
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [27]:
# pd.set_option('display.max_colwidth', None)

## 1. Ingestion

### 1.1. (Used in Evaluator)Semantic-Type Negative Sampling dict('m:umls term', 'sematic-type') 400K

In [17]:
with open(
    '/lustre/acslab/shared/Agatha_shared/2021_11_22/2021_11_22_semtypes_to_nodelbl_and_vv_dict.json', 'r'
) as f:
    st_nodelbl_dict = json.load(f)

In [18]:
for i, (key, value) in enumerate(st_nodelbl_dict.items()):
    print(f"{key}: {value}")
    if i == 8: 
        break

m:c0000005: Pharmacologic Substance
m:c0000039: Organic Chemical
m:c0000052: Amino Acid, Peptide, or Protein
m:c0000084: Amino Acid, Peptide, or Protein
m:c0000096: Organic Chemical
m:c0000097: Organic Chemical
m:c0000098: Organic Chemical
m:c0000102: Organic Chemical
m:c0000103: Organic Chemical


In [19]:
len(st_nodelbl_dict)

414961

### 1.2. PredicatorGraph -> a Subset of Semantic Graph with only UMLS term and Predicates

In [20]:
workingFolder = \
    '/lustre/acslab/shared/Agatha_shared/2021_11_22'

model_path = f'{workingFolder}/model_epoch5_darwin.pt'
embedding_path = f'{workingFolder}/embeddings/predicate_subset'
entity_db = f'{workingFolder}/predicate_entities.sqlite3'
graph_db = f'{workingFolder}/predicate_graph.sqlite3'

In [21]:
model = torch.load(model_path)

model.configure_paths(
  embedding_dir=embedding_path,
  entity_db=entity_db,
  graph_db=graph_db,
)

model = model.eval()

In [22]:
model.predict_from_terms([('m:c0026549', 'm:c5203670')])

[0.8342379331588745]

In [23]:
# subset of semantic graph, we cal predictor graph
nodelist = model.graph.keys()

In [24]:
len(nodelist)

26953485

In [25]:
pred_pairs_list = [
    p for p in tqdm(nodelist) if p[0] == 'p'
]

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 26953485/26953485 [00:10<00:00, 2458900.39it/s]


In [26]:
pred_cui_pairs_set = set()

for p in tqdm(pred_pairs_list):
    p_split = p.upper().split(':')
    
    s = p_split[1]
    o = p_split[-1]
    
    pred_cui_pairs_set.add(
        tuple(sorted([s,o]))
    )

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 26535915/26535915 [00:46<00:00, 575952.30it/s]


In [27]:
len(pred_cui_pairs_set)

19086440

In [28]:
# pred_cui_pairs_set
# {('C0011860', 'C3880361'),
#  ('C0008942', 'C0431128'),
#  ('C0024880', 'C0281255'),
#  ('C0229671', 'C0534397'),
#  ('C0028633', 'C1826697'),
#  ('C0015934', 'C0241910'),
#  ('C0022742', 'C0851881'),

### 1.3. SemrepHandler ==> Identify Predicates

In [64]:
nlm_soft_folder = '/lustre/acslab/users/3281/SemRep'
sr_temp_folder = '/lustre/acslab/users/3281/semrep_temp_4'
sr_replace_utf8_path = '/lustre/acslab/users/3281/SemRep/replace_utf8.jar'

In [65]:
t = SemRepHandler(
    nlm_soft_path=nlm_soft_folder,
    temp_folder=sr_temp_folder,
    replace_utf8_path=sr_replace_utf8_path,
)

In [66]:
t.sr_binary_path = Path('/lustre/acslab/users/3281/SemRep/public_semrep/bin/semrep.v1.9_2021AB')

In [67]:
t.ProcessList_parallel(['morphine and covid 19'])

Run SemRep in interactive mode...
Processing input with replace_utf8.jar utility...


{'s:user_input_0:1': {'sent_text': 'morphine and covid 19 ',
  'terms': [{'CID': 'C0026549',
    'pref_name': 'morphine',
    'extracted_text': 'morphine',
    'label': 'UMLS',
    'sem_types': ['orch', 'phsu'],
    'negated': False},
   {'CID': 'C5203670',
    'pref_name': 'COVID-19',
    'extracted_text': 'covid 19',
    'label': 'UMLS',
    'sem_types': ['dsyn'],
    'negated': False}],
  'relations': []}}

In [18]:
pd.DataFrame(t.ProcessList_parallel(['morphine and covid 19'])['s:user_input_0:1']['terms'])

Run SemRep in interactive mode...
Processing input with replace_utf8.jar utility...


,CID,pref_name,extracted_text,label,sem_types,negated
0,C0026549,morphine,morphine,UMLS,"[orch, phsu]",False
1,C5203670,COVID-19,covid 19,UMLS,[dsyn],False


In [19]:
pd.DataFrame(t.ProcessList_parallel(['morphine and covid 19'])['s:user_input_0:1']['relations'])

Run SemRep in interactive mode...
Processing input with replace_utf8.jar utility...


""


## 2. Retrieval paths from Co-occurring Graph

In [31]:
df = pd.read_pickle('pmid_cuis_bench_cooc_2021_only_agg_gt2_mini_df.pkl')

In [32]:
df

,pair,abstr_lvl_cooc,abstr_lvl_cooc_len,source,target,source_st,target_st,source_name,target_name,context,context_len,sp_w_pmids
13,C0002598 C0025153,"{32493475, 32493478, 32616067}",3,C0002598,C0025153,"{Pharmacologic Substance, Organic Chemical}","{Pharmacologic Substance, Organic Chemical}",amiodarone,mefloquine,[Efficacy of hydroxychloroquine for post-expos...,3,"{('C0002598', 'C0002641', 'C0025153'): {'C0002..."
224,C0026549 C5203670,"{32588748, 32839778, 33612113, 33331392, 32387...",16,C0026549,C5203670,"{Pharmacologic Substance, Organic Chemical}",{Disease or Syndrome},morphine,COVID19 (disease),[An audit of end-of-life symptom control in pa...,16,"{('C0026549', 'C0021760', 'C5203670'): {'C0021..."
296,C0039286 C5203670,"{32663742, 32493494, 33651271, 33855640}",4,C0039286,C5203670,"{Pharmacologic Substance, Organic Chemical}",{Disease or Syndrome},tamoxifen,COVID19 (disease),"[Controlled, double-blind, randomized trial to...",4,"{('C5203670', 'C0052796', 'C0039286'): {'C0052..."
521,C0812198 C1412096,"{34171401, 33149621, 32244545}",3,C0812198,C1412096,{Gene or Genome},{Gene or Genome},BAX gene,ABCG1 gene,"[1, 6-O, O-Diacetylbritannilactone from Inula ...",3,"{('C0812198', 'C0001047', 'C1412096'): {'C0001..."
711,C1419407 C1565830,"{32740646, 32882325, 33133130, 34249481}",4,C1419407,C1565830,{Gene or Genome},"{Amino Acid, Peptide, or Protein, Enzyme}",RNASE1 gene,"PTGS1 protein, human",[Development and validation of a six-RNA bindi...,4,"{('C1565830', 'C1414968', 'C1419407'): {'C1414..."


### 2.1. Number of Paths between Morphine + Covid 19 => 15

### 2.2. Reference: future abstract:
we have 16 abstracts as refewrence, 1) as initial step we just summarize them. kept as a reference for Correctmess Evaluation. 2) you can use find-k-top func to choose among 16 to find best match among them.

In [33]:
df['context'][224]

['An audit of end-of-life symptom control in patients with corona virus disease 2019 (COVID-19) dying in a hospital in the United Kingdom. The literature contains limited information on the problems faced by dying patients with COVID-19 and the effectiveness of interventions to manage these. The aim of this audit was to assess the utility of our end-of-life care plan, and specifically the effectiveness of our standardised end-of-life care treatment algorithms, in dying patients with COVID-19. The audit primarily involved data extraction from the end-of-life care plan, which includes four hourly nursing (ward nurses) assessments of specific problems: patients with problems were managed according to standardised treatment algorithms, and the intervention was deemed to be effective if the problem was not present at subsequent assessments. This audit was undertaken at a general hospital in England, covered the 8 weeks from 16 March to 11 May 2020 and included all inpatients with COVID-19 w

Here is summarized version of Reference for morphine-covid19 path:

###
These abstracts explore the challenges and strategies related to COVID-19 symptom management, end-of-life care, and treatment adaptations. 
Several studies emphasize the effectiveness of opioids like morphine and fentanyl for symptom control, particularly breathlessness and agitation, in end-of-life COVID-19 care. 
However, delays in cancer care and restricted healthcare access have led to increased self-medication and psychosocial distress among patients, especially in disadvantaged communities. 
Structural studies of the mu-opioid receptor reveal fentanyl's unique binding mechanisms, which could inform safer analgesic design. 
Immunomodulatory effects of opioids and ACE2 inhibition are highlighted as potential therapeutic strategies against COVID-19. 
Despite lockdown challenges, tele-consultation and home-based palliative care have maintained continuity in patient management. 
The REACT-SCOT study links severe COVID-19 with polypharmacy and specific drug classes, advocating for reducing overprescribing. 
Lastly, scaffold morphing of arbidol offers multi-targeting potential, while COVID-19
psychological impact on methadone patients underscores the need for enhanced mental health support.
###

In [34]:
df['sp_w_pmids'][224]

{('C0026549',
  'C0021760',
  'C5203670'): {'C0021760 C0026549': ['Effect of intravenous pentoxifylline in inflammatory response in patients undergoing nephrolithotomy. To determine the potential efficacy of intravenous (IV) infusion of pentoxifylline (PTX) before nephrolithotomy on attenuating plasma level of the tumor necrosis factor (TNF)-alpha and interleukin (IL)-1, and to investigate whether it prevents postoperative pain. In a randomized, double-blind, placebo-controlled study, 32 patients (American Society of Anesthesiologists physical status 1 and 2) who were undergoing general anesthesia for nephrolithotomy were randomized to receive intravenous PTX (500 mg in 500 mL saline for 2 hours followed by 700 mg in 1000 mL saline for 6 hours), or placebo (1500 mL saline) before induction of the anesthesia. Two venous blood samples were obtained 10 minutes before PTX or placebo infusion and after surgery at 24 hours for laboratory examination. After surgery, the amount of narcotics co

In [35]:
morphine_covid19_paths = df['sp_w_pmids'][224].keys()
morphine_covid19_paths

dict_keys([('C0026549', 'C0021760', 'C5203670'), ('C0026549', 'C0021756', 'C5203670'), ('C0026549', 'C0085295', 'C5203670'), ('C0026549', 'C1448177', 'C5203670'), ('C0026549', 'C0042679', 'C5203670'), ('C0026549', 'C0011777', 'C5203670'), ('C0026549', 'C0014563', 'C5203670'), ('C0026549', 'C0009079', 'C5203670'), ('C0026549', 'C2976303', 'C5203670'), ('C0026549', 'C0006560', 'C5203670'), ('C0026549', 'C0008269', 'C5203670'), ('C0026549', 'C0085149', 'C5203670'), ('C0026549', 'C0209368', 'C5203670'), ('C0026549', 'C0020336', 'C5203670'), ('C0026549', 'C1609165', 'C5203670')])

In [36]:
num_path_morphine_Covid = len(df['sp_w_pmids'][224].keys())
num_path_morphine_Covid

15

In [37]:
df['sp_w_pmids'][224][('C0026549', 'C0021760', 'C5203670')].keys()

dict_keys(['C0021760 C0026549', 'C0021760 C5203670'])

### 2.3. Take one Path 

In [38]:
path1_C0021760_CGF = df['sp_w_pmids'][224][('C0026549', 'C0021760', 'C5203670')].values()
path1_C0021760_CGF


dict_values([['Effect of intravenous pentoxifylline in inflammatory response in patients undergoing nephrolithotomy. To determine the potential efficacy of intravenous (IV) infusion of pentoxifylline (PTX) before nephrolithotomy on attenuating plasma level of the tumor necrosis factor (TNF)-alpha and interleukin (IL)-1, and to investigate whether it prevents postoperative pain. In a randomized, double-blind, placebo-controlled study, 32 patients (American Society of Anesthesiologists physical status 1 and 2) who were undergoing general anesthesia for nephrolithotomy were randomized to receive intravenous PTX (500 mg in 500 mL saline for 2 hours followed by 700 mg in 1000 mL saline for 6 hours), or placebo (1500 mL saline) before induction of the anesthesia. Two venous blood samples were obtained 10 minutes before PTX or placebo infusion and after surgery at 24 hours for laboratory examination. After surgery, the amount of narcotics consumption and intensity of pain (Visual Analog Scale

### 2.4. path1 abstract

In [39]:
dict_values_abstracts = df['sp_w_pmids'][224][('C0026549', 'C0021760', 'C5203670')].values()

path1_C0021760_CGF = [abstract for sublist in dict_values_abstracts for abstract in sublist]
path1_C0021760_CGF

['Effect of intravenous pentoxifylline in inflammatory response in patients undergoing nephrolithotomy. To determine the potential efficacy of intravenous (IV) infusion of pentoxifylline (PTX) before nephrolithotomy on attenuating plasma level of the tumor necrosis factor (TNF)-alpha and interleukin (IL)-1, and to investigate whether it prevents postoperative pain. In a randomized, double-blind, placebo-controlled study, 32 patients (American Society of Anesthesiologists physical status 1 and 2) who were undergoing general anesthesia for nephrolithotomy were randomized to receive intravenous PTX (500 mg in 500 mL saline for 2 hours followed by 700 mg in 1000 mL saline for 6 hours), or placebo (1500 mL saline) before induction of the anesthesia. Two venous blood samples were obtained 10 minutes before PTX or placebo infusion and after surgery at 24 hours for laboratory examination. After surgery, the amount of narcotics consumption and intensity of pain (Visual Analog Scale and Verbal R

In [19]:
paths_values = {}
# Iterate over each key in the dictionary
for key in df['sp_w_pmids'][224].keys():
    values_list = list(df['sp_w_pmids'][224][key].values())
    paths_values[key] = values_list

for path, values in paths_values.items():
    print(f"Path: {path} -> Values: {values}")


Path: ('C0026549', 'C0021760', 'C5203670') -> Values: [['Effect of intravenous pentoxifylline in inflammatory response in patients undergoing nephrolithotomy. To determine the potential efficacy of intravenous (IV) infusion of pentoxifylline (PTX) before nephrolithotomy on attenuating plasma level of the tumor necrosis factor (TNF)-alpha and interleukin (IL)-1, and to investigate whether it prevents postoperative pain. In a randomized, double-blind, placebo-controlled study, 32 patients (American Society of Anesthesiologists physical status 1 and 2) who were undergoing general anesthesia for nephrolithotomy were randomized to receive intravenous PTX (500 mg in 500 mL saline for 2 hours followed by 700 mg in 1000 mL saline for 6 hours), or placebo (1500 mL saline) before induction of the anesthesia. Two venous blood samples were obtained 10 minutes before PTX or placebo infusion and after surgery at 24 hours for laboratory examination. After surgery, the amount of narcotics consumption 

In [22]:
paths_values = {}

for key in df['sp_w_pmids'][224].keys():
    values_list = list(df['sp_w_pmids'][224][key].values())
    paths_values[key] = values_list

max_length = max([len(v) for v in paths_values.values()])
for key in paths_values.keys():
    if len(paths_values[key]) < max_length:
        paths_values[key].extend([None] * (max_length - len(paths_values[key])))

paths_df = pd.DataFrame.from_dict(paths_values, orient='index')
paths_df.columns = [f"Value_{i+1}" for i in range(paths_df.shape[1])]

paths_df.reset_index(inplace=True)
paths_df.rename(columns={'index': 'Path'}, inplace=True)
paths_df

,Path,Value_1,Value_2
0,"(C0026549, C0021760, C5203670)",[Effect of intravenous pentoxifylline in infla...,[Simple predictive models identify patients wi...
1,"(C0026549, C0021756, C5203670)",[Impact of HACA on Immunomodulation and Treatm...,[Continuous tracking of COVID-19 patients' imm...
2,"(C0026549, C0085295, C5203670)",[Differential involvement of RelB in morphine-...,[Analysis of cytokines in SARS-CoV-2 or COVID-...
3,"(C0026549, C1448177, C5203670)",[Withdrawal from morphine in mice suppresses s...,[Inflammation and kidney involvement in human ...
4,"(C0026549, C0042679, C5203670)",[Impact of anti-cancer drugs and other determi...,[Lymphocyte-to-C-Reactive Protein Ratio: A Nov...
5,"(C0026549, C0011777, C5203670)","[2-Allylphenol Reduces IL-1beta and TNF-alpha,...",[Essential medicine selection during the COVID...
6,"(C0026549, C0014563, C5203670)",[A combined chest drain and intrapleural cathe...,[Extracorporeal Membrane Oxygenation for COVID...
7,"(C0026549, C0009079, C5203670)",[Olanzapine- and clozapine-induced stuttering....,[Psychoses by Attacks from Subverted Mast Cell...
8,"(C0026549, C2976303, C5203670)",[Direct-acting antiviral interactions with opi...,[Small-molecule antiviral agents in ongoing cl...
9,"(C0026549, C0006560, C5203670)",[Intravenous Dexamethasone Injection Reduces P...,[Diabetes mellitus as a comorbidity in COVID-1...


In [68]:
morphine_covid19_paths_items = df['sp_w_pmids'][224].items()
morphine_covid19_paths_items

dict_items([(('C0026549', 'C0021760', 'C5203670'), {'C0021760 C0026549': ['Effect of intravenous pentoxifylline in inflammatory response in patients undergoing nephrolithotomy. To determine the potential efficacy of intravenous (IV) infusion of pentoxifylline (PTX) before nephrolithotomy on attenuating plasma level of the tumor necrosis factor (TNF)-alpha and interleukin (IL)-1, and to investigate whether it prevents postoperative pain. In a randomized, double-blind, placebo-controlled study, 32 patients (American Society of Anesthesiologists physical status 1 and 2) who were undergoing general anesthesia for nephrolithotomy were randomized to receive intravenous PTX (500 mg in 500 mL saline for 2 hours followed by 700 mg in 1000 mL saline for 6 hours), or placebo (1500 mL saline) before induction of the anesthesia. Two venous blood samples were obtained 10 minutes before PTX or placebo infusion and after surgery at 24 hours for laboratory examination. After surgery, the amount of narc

### 2.5. Use SBERT to find top k closest

In [40]:
len(path1_C0021760_CGF)

6

In [41]:
model_sbert = SentenceTransformer('all-MiniLM-L6-v2')
query = "COVID-19 Virus Disease and morphine"
documents = path1_C0021760_CGF

def find_top_documents(query, contexts, top_k=1, return_scores=True):
    query_embedding = model_sbert.encode(query, convert_to_tensor=True)
    context_embeddings = model_sbert.encode(contexts, convert_to_tensor=True)

    search_results = util.semantic_search(query_embedding, context_embeddings, top_k=top_k)[0]
    top_documents = [contexts[hit['corpus_id']] for hit in search_results]

    if return_scores:
        top_scores = [hit['score'] for hit in search_results]
        return list(zip(top_documents, top_scores))
    else:
        return top_documents

top_results = find_top_documents(query, documents, top_k=6, return_scores=True)
print(top_results)

[('Early Viral Clearance and Antibody Kinetics of COVID-19 Among Asymptomatic Carriers. Asymptomatic carriers contribute to the spread of Coronavirus Disease 2019 (COVID-19), but their clinical characteristics, viral kinetics, and antibody responses remain unclear. A total of 56 COVID-19 patients without symptoms at admission and 19 age-matched symptomatic patients were enrolled. RNA of SARS-CoV-2 was tested using transcriptase quantitative PCR, and the total antibodies (Ab), IgG, IgA, and IgM against the SARS-CoV-2 were tested using Chemiluminescence Microparticle Immuno Assay. Among 56 patients without symptoms at admission, 33 cases displayed symptoms and 23 remained asymptomatic throughout the follow-up period. 43.8% of the asymptomatic carriers were children and none of the asymptomatic cases had recognizable changes in C-reactive protein or interleukin-6, except one 64-year-old patient. The initial threshold cycle value of nasopharyngeal SARS-CoV-2 in asymptomatic carriers was si

In [55]:
query = "COVID-19 Virus Disease and morphine"
results_path1 = find_top_documents(query, documents, top_k=6, return_scores=False)
results_path1

['Early Viral Clearance and Antibody Kinetics of COVID-19 Among Asymptomatic Carriers. Asymptomatic carriers contribute to the spread of Coronavirus Disease 2019 (COVID-19), but their clinical characteristics, viral kinetics, and antibody responses remain unclear. A total of 56 COVID-19 patients without symptoms at admission and 19 age-matched symptomatic patients were enrolled. RNA of SARS-CoV-2 was tested using transcriptase quantitative PCR, and the total antibodies (Ab), IgG, IgA, and IgM against the SARS-CoV-2 were tested using Chemiluminescence Microparticle Immuno Assay. Among 56 patients without symptoms at admission, 33 cases displayed symptoms and 23 remained asymptomatic throughout the follow-up period. 43.8% of the asymptomatic carriers were children and none of the asymptomatic cases had recognizable changes in C-reactive protein or interleukin-6, except one 64-year-old patient. The initial threshold cycle value of nasopharyngeal SARS-CoV-2 in asymptomatic carriers was sim

In [46]:
len(results_path1)

6

### 2.6. Full explanation with oai_get_response from LLM (GPT4)

In [47]:
llm_model = "gpt-4-0125-preview"

In [49]:
client = OpenAI(
    api_key=open('OAI_CONFIG_LIST').read().strip(),
)

In [50]:
chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "Hi, how are you?",
        }
    ],
    model=llm_model,
    temperature=0.5,
)

In [51]:
chat_completion.choices[0].message.content

"I'm just a computer program, so I don't have feelings, but thanks for asking! How can I help you today?"

In [52]:
def oai_get_response(msg, temp):
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": msg,
            }
        ],
        model=llm_model,
        temperature=temp,
    )
    
    reply_text = chat_completion.choices[0].message.content
    
    return reply_text

### 2.7. Generate Prompt text with new Retrieval + Context

In [56]:
results_path1

['Early Viral Clearance and Antibody Kinetics of COVID-19 Among Asymptomatic Carriers. Asymptomatic carriers contribute to the spread of Coronavirus Disease 2019 (COVID-19), but their clinical characteristics, viral kinetics, and antibody responses remain unclear. A total of 56 COVID-19 patients without symptoms at admission and 19 age-matched symptomatic patients were enrolled. RNA of SARS-CoV-2 was tested using transcriptase quantitative PCR, and the total antibodies (Ab), IgG, IgA, and IgM against the SARS-CoV-2 were tested using Chemiluminescence Microparticle Immuno Assay. Among 56 patients without symptoms at admission, 33 cases displayed symptoms and 23 remained asymptomatic throughout the follow-up period. 43.8% of the asymptomatic carriers were children and none of the asymptomatic cases had recognizable changes in C-reactive protein or interleukin-6, except one 64-year-old patient. The initial threshold cycle value of nasopharyngeal SARS-CoV-2 in asymptomatic carriers was sim

In [57]:
llm_fix_prompt= 'How would you describe an indirect relationship between COVID-19 Virus Disease and morphine given the following scientific abstracts as context?'
llm_fix_prompt_str = '\n\n'.join(results_path1)
llm_fix_prompt_combo = llm_fix_prompt + '\n' + llm_fix_prompt_str
llm_fix_prompt_combo

"How would you describe an indirect relationship between COVID-19 Virus Disease and morphine given the following scientific abstracts as context?\nEarly Viral Clearance and Antibody Kinetics of COVID-19 Among Asymptomatic Carriers. Asymptomatic carriers contribute to the spread of Coronavirus Disease 2019 (COVID-19), but their clinical characteristics, viral kinetics, and antibody responses remain unclear. A total of 56 COVID-19 patients without symptoms at admission and 19 age-matched symptomatic patients were enrolled. RNA of SARS-CoV-2 was tested using transcriptase quantitative PCR, and the total antibodies (Ab), IgG, IgA, and IgM against the SARS-CoV-2 were tested using Chemiluminescence Microparticle Immuno Assay. Among 56 patients without symptoms at admission, 33 cases displayed symptoms and 23 remained asymptomatic throughout the follow-up period. 43.8% of the asymptomatic carriers were children and none of the asymptomatic cases had recognizable changes in C-reactive protein 

### 2.8. LLM generate response 

In [59]:
llm_resp_path1 = oai_get_response(llm_fix_prompt_combo,0)
llm_resp_path1

"The indirect relationship between COVID-19 Virus Disease and morphine can be inferred through the interplay of immune response and the potential impact of morphine on the central nervous system (CNS) and immune function. Here's a synthesis of the provided abstracts to illustrate this relationship:\n\n1. **COVID-19 Immune Response**: The abstracts on COVID-19 highlight the importance of the immune response in managing the disease, particularly the role of cytokines like IL-6 in the severity of COVID-19 pneumonia and the cytokine storm. Tocilizumab, an IL-6 receptor antibody, has been shown to mitigate the cytokine storm, suggesting that modulation of the immune response is crucial in the treatment of COVID-19.\n\n2. **Morphine's Effect on the Brain and Immune System**: Morphine is known to induce epigenetic changes in the brain, affecting genes involved in brain function and the immune response, such as Il6. Morphine's ability to cross the blood-brain barrier (BBB) and affect brain fun

## 3. EVALUATION: Eval each sentence according Agatha Rank

In [60]:
def generate_negatives(pos_pair, sample_rate=5):
    
    s = pos_pair[0]
    o = pos_pair[1]
    
    if s[0] != 'm':
        s = f'm:{pos_pair[0].lower()}'
    
    if o[0] != 'm':
        o = f'm:{pos_pair[1].lower()}'
    
    if o in st_nodelbl_dict:
        o_st = st_nodelbl_dict[o]
        o_sample_list = random.sample(
            st_nodelbl_dict[o_st],
            sample_rate
        )
    else:
        o_sample_list = []
    
    out_pairs = []
    for o_neg in o_sample_list:
        out_pairs.append((s, o_neg))
        
    return out_pairs

In [61]:
def eval_pair(pair, sample_rate = 10):
    
    pair_negs = generate_negatives(pair, sample_rate=sample_rate)
    
    agatha_queries = [pair] + pair_negs
    
    pair_labels = [1] + [0]*len(pair_negs)
    
    scores = model.predict_from_terms(agatha_queries)
    
    res_list = sorted(
        list(zip(scores, pair_labels)),
        key=lambda x: x[0],
        reverse=True
    )
    
    rank = None
    for i, (score, lbl) in enumerate(res_list):
        if lbl == 1:
            rank = i
            break
    
    return {
        'scores': res_list,
        'pos_rank': rank + 1
    }

In [62]:
# EvaluateLLM: if sub,obj exist in Agatha, then based on condition of score out of Agatha is less than Threshold(0.8) mark as bad prediction
def evaluate_llm_resp(llm_resp_str):
    print("This is original generated LLM res: " + llm_resp_str)

    expl_parts_list = []

    cleaned_text = llm_resp_str.replace('\n', '').replace('*', '')

    print("This is CLEANED generated LLM res :" + cleaned_text)

    print('\n-----------------------------------------------\n')
    
    expl_sr_out = t.ProcessList_parallel([cleaned_text], nthreads=10)
    
    # print('\n-----------------------------------------------\n')

    for s_id, sent_data in expl_sr_out.items():
        sent_text = sent_data['sent_text']

        status = 'ok'
        bad_predicates = []
        good_predicates = []
        for rel in sent_data['relations']:

            rel_subj = rel['subj_text']
            v = rel['verb']
            rel_obj = rel['obj_text']

            #pred = [rel_subj, v, rel_obj]
            pred = [rel_subj, rel_obj]

            rel_pair = tuple(
                sorted(
                    [
                        rel['subj_id'],
                        rel['obj_id']
                    ]
                )
            )
            if rel_pair in pred_cui_pairs_set:
                good_predicates.append(pred)
            if rel_pair not in pred_cui_pairs_set:
                #status = 'REWORK'
                subj_id = f"m:{rel['subj_id'].lower()}"
                obj_id = f"m:{rel['obj_id'].lower()}"
                if subj_id in model.graph and obj_id in model.graph:
                    agatha_score = model.predict_from_terms([[subj_id, obj_id]])[0]
                    agatha_rank = eval_pair([subj_id, obj_id], sample_rate=20)['pos_rank']
                    pred.append(agatha_rank)
                    # if agatha_score < 0.9:
                    if agatha_rank > 1:
                        status = 'REWORK'
                        bad_predicates.append(pred)
                        print("Agatha score for predicate: {}".format(agatha_score))
                    else:
                        good_predicates.append(pred)


        print('Sentence:\n', sent_text)
        print('Status:\n', status)
        if status != 'ok':
            print('Reason:')
            for p in bad_predicates:
                print('\t', p, 'not in AGATHA KB')
        else:
            if len(good_predicates):
                print('Recognized:')
            for p in good_predicates:
                print('\t', p, 'in AGATHA KB')

        print('\n-----\n')

        expl_parts_list.append(
            {
                'sent_text': sent_text,
                'status': status,
                'bad_predicates': bad_predicates,
                'good_predicates': good_predicates
            }
        )
    print(expl_parts_list)    
    return expl_parts_list

### 3.1. First Evaluate llm response for indirect connection

### 3.1.1 Iteration 1

In [68]:
evaluation_result = evaluate_llm_resp(llm_resp_path1)

This is original generated LLM res: The indirect relationship between COVID-19 Virus Disease and morphine can be inferred through the interplay of immune response and the potential impact of morphine on the central nervous system (CNS) and immune function. Here's a synthesis of the provided abstracts to illustrate this relationship:

1. **COVID-19 Immune Response**: The abstracts on COVID-19 highlight the importance of the immune response in managing the disease, particularly the role of cytokines like IL-6 in the severity of COVID-19 pneumonia and the cytokine storm. Tocilizumab, an IL-6 receptor antibody, has been shown to mitigate the cytokine storm, suggesting that modulation of the immune response is crucial in the treatment of COVID-19.

2. **Morphine's Effect on the Brain and Immune System**: Morphine is known to induce epigenetic changes in the brain, affecting genes involved in brain function and the immune response, such as Il6. Morphine's ability to cross the blood-brain bar

In [69]:
expl_parts_df = pd.DataFrame(evaluation_result).reset_index()
expl_parts_df['reworked_part'] = None
expl_parts_df

,index,sent_text,status,bad_predicates,good_predicates,reworked_part
0,0,The indirect relationship between COVID-19 Vir...,ok,[],"[[impact, COVID-19 Virus Disease]]",None
1,1,Here's a synthesis of the provided abstracts t...,ok,[],[],None
2,2,COVID-19 Immune Response: The abstracts on COV...,ok,[],[],None
3,3,"Tocilizumab, an IL-6 receptor antibody, has be...",ok,[],"[[Tocilizumab, cytokine storm]]",None
4,4,Morphine's Effect on the Brain and Immune Syst...,ok,[],"[[Morphine, epigenetic changes], [genes, immun...",None
5,5,Morphine's ability to cross the blood-brain ba...,ok,[],"[[blood-brain barrier, brain function]]",None
6,6,Cytokines and the Blood-Brain Barrier: The pas...,ok,[],"[[BBB, cytokines], [cytokines, CNS]]",None
7,7,Given that morphine can affect the CNS and has...,ok,[],"[[morphine, immune response]]",None
8,8,Indirect Relationship: The indirect relationsh...,ok,[],[],None
9,9,While COVID-19's severity is influenced by the...,ok,[],[],None


In [70]:
expl_parts_enum_dict = dict(enumerate(evaluation_result))

In [71]:
max(expl_parts_enum_dict)

11

### 3.2. Find out from which Abstract Rework sentence of explanation is comming from.

In [72]:
expl_parts_enum_dict = dict(enumerate(evaluation_result))
rework_sentences = []

for key, value in expl_parts_enum_dict.items():
    if value['status'] == 'REWORK':  
        rework_sentences.append(value['sent_text']) 
print(rework_sentences)

["This could be through altering the expression of genes involved in the immune response, affecting cytokine levels, or influencing the passage of cytokines across the BBB.In summary, while there is no direct link between morphine use and COVID-19 outcomes, the potential for morphine to influence the immune response and CNS function indirectly suggests a complex interplay that could impact the body's response to COVID-19. "]


In [73]:
rework_sent_to_drop_abst_1 = rework_sentences[0]
rework_sent_to_drop_abst_1

"This could be through altering the expression of genes involved in the immune response, affecting cytokine levels, or influencing the passage of cytokines across the BBB.In summary, while there is no direct link between morphine use and COVID-19 outcomes, the potential for morphine to influence the immune response and CNS function indirectly suggests a complex interplay that could impact the body's response to COVID-19. "

### 3.3. find closest abstracts to Rework_sentence ==> by querying db 

In [75]:
cur_top_n_context_abstr_drop_list_1 = find_top_documents(rework_sent_to_drop_abst_1,results_path1, top_k=1,return_scores=False)
cur_top_n_context_abstr_drop_list_1

['The effect of morphine upon DNA methylation in ten regions of the rat brain. Morphine is one of the most effective analgesics in medicine. However, its use is associated with the development of tolerance and dependence. Recent studies demonstrating epigenetic changes in the brain after exposure to opiates have provided insight into mechanisms possibly underlying addiction. In this study, we sought to identify epigenetic changes in ten regions of the rat brain following acute and chronic morphine exposure. We analyzed DNA methylation of six nuclear-encoded genes implicated in brain function (Bdnf, Comt, Il1b, Il6, Nr3c1, and Tnf) and three mitochondrially-encoded genes (Mtco1, Mtco2, and Mtco3), and measured global 5-methylcytosine (5mC) and 5-hydroxymethylcytosine (5 hmC) levels. We observed differential methylation of Bdnf and Il6 in the pons, Nr3c1 in the cerebellum, and Il1b in the hippocampus in response to acute morphine exposure (all P value < 0.05). Chronic exposure was associ

### 3.4. Drop that abstract from Context

In [77]:
results_path1[1]

'The effect of morphine upon DNA methylation in ten regions of the rat brain. Morphine is one of the most effective analgesics in medicine. However, its use is associated with the development of tolerance and dependence. Recent studies demonstrating epigenetic changes in the brain after exposure to opiates have provided insight into mechanisms possibly underlying addiction. In this study, we sought to identify epigenetic changes in ten regions of the rat brain following acute and chronic morphine exposure. We analyzed DNA methylation of six nuclear-encoded genes implicated in brain function (Bdnf, Comt, Il1b, Il6, Nr3c1, and Tnf) and three mitochondrially-encoded genes (Mtco1, Mtco2, and Mtco3), and measured global 5-methylcytosine (5mC) and 5-hydroxymethylcytosine (5 hmC) levels. We observed differential methylation of Bdnf and Il6 in the pons, Nr3c1 in the cerebellum, and Il1b in the hippocampus in response to acute morphine exposure (all P value < 0.05). Chronic exposure was associa

In [78]:
results_path1.pop(1)
results_path1

['Early Viral Clearance and Antibody Kinetics of COVID-19 Among Asymptomatic Carriers. Asymptomatic carriers contribute to the spread of Coronavirus Disease 2019 (COVID-19), but their clinical characteristics, viral kinetics, and antibody responses remain unclear. A total of 56 COVID-19 patients without symptoms at admission and 19 age-matched symptomatic patients were enrolled. RNA of SARS-CoV-2 was tested using transcriptase quantitative PCR, and the total antibodies (Ab), IgG, IgA, and IgM against the SARS-CoV-2 were tested using Chemiluminescence Microparticle Immuno Assay. Among 56 patients without symptoms at admission, 33 cases displayed symptoms and 23 remained asymptomatic throughout the follow-up period. 43.8% of the asymptomatic carriers were children and none of the asymptomatic cases had recognizable changes in C-reactive protein or interleukin-6, except one 64-year-old patient. The initial threshold cycle value of nasopharyngeal SARS-CoV-2 in asymptomatic carriers was sim

In [79]:
len(results_path1)

5

## 4. Feedback Loop Action: -> Regenerate Explanation from LLM after Drop Rework_abstrac from Context:

#### 4.1. generate a new sentence processing prompt (Donot include any history of rework sentence)

#### Iteration 2

In [80]:
llm_fix_prompt= 'How would you describe an indirect relationship between Covid Virus disease and morphine the following scientific abstracts as context?'
llm_fix_prompt_str = '\n\n'.join(results_path1)
llm_fix_prompt_combo = llm_fix_prompt + '\n' + llm_fix_prompt_str
llm_fix_prompt_combo

"How would you describe an indirect relationship between Covid Virus disease and morphine the following scientific abstracts as context?\nEarly Viral Clearance and Antibody Kinetics of COVID-19 Among Asymptomatic Carriers. Asymptomatic carriers contribute to the spread of Coronavirus Disease 2019 (COVID-19), but their clinical characteristics, viral kinetics, and antibody responses remain unclear. A total of 56 COVID-19 patients without symptoms at admission and 19 age-matched symptomatic patients were enrolled. RNA of SARS-CoV-2 was tested using transcriptase quantitative PCR, and the total antibodies (Ab), IgG, IgA, and IgM against the SARS-CoV-2 were tested using Chemiluminescence Microparticle Immuno Assay. Among 56 patients without symptoms at admission, 33 cases displayed symptoms and 23 remained asymptomatic throughout the follow-up period. 43.8% of the asymptomatic carriers were children and none of the asymptomatic cases had recognizable changes in C-reactive protein or interl

In [81]:
%%time
llm_resp_reworked = oai_get_response(llm_fix_prompt_combo,0)

CPU times: user 17.5 ms, sys: 13.9 ms, total: 31.4 ms
Wall time: 38.1 s


In [82]:
llm_resp_reworked

"The indirect relationship between COVID-19 and morphine can be inferred through the understanding of the inflammatory response associated with COVID-19 and the potential impact of morphine on the central nervous system (CNS) via the blood-brain barrier (BBB). The scientific abstracts provided offer insights into various aspects of COVID-19, including its asymptomatic spread, predictive models for severe outcomes, the role of cytokines in the disease's pathology, and the effects of certain treatments on inflammatory markers.\n\nCOVID-19 has been shown to induce a significant inflammatory response in the body, characterized by elevated levels of cytokines such as IL-6 and TNF-alpha. This cytokine storm is a hallmark of severe COVID-19 cases and can lead to critical outcomes. The abstracts discuss the role of cytokines in COVID-19 and explore treatments aimed at reducing this inflammatory response, such as the use of Tocilizumab, an IL-6 receptor antibody.\n\nMorphine's potential indirec

In [83]:
evaluation_result = evaluate_llm_resp(llm_resp_reworked)

This is original generated LLM res: The indirect relationship between COVID-19 and morphine can be inferred through the understanding of the inflammatory response associated with COVID-19 and the potential impact of morphine on the central nervous system (CNS) via the blood-brain barrier (BBB). The scientific abstracts provided offer insights into various aspects of COVID-19, including its asymptomatic spread, predictive models for severe outcomes, the role of cytokines in the disease's pathology, and the effects of certain treatments on inflammatory markers.

COVID-19 has been shown to induce a significant inflammatory response in the body, characterized by elevated levels of cytokines such as IL-6 and TNF-alpha. This cytokine storm is a hallmark of severe COVID-19 cases and can lead to critical outcomes. The abstracts discuss the role of cytokines in COVID-19 and explore treatments aimed at reducing this inflammatory response, such as the use of Tocilizumab, an IL-6 receptor antibody

In [84]:
expl_parts_df = pd.DataFrame(evaluation_result).reset_index()
expl_parts_df['reworked_part'] = None
expl_parts_df

,index,sent_text,status,bad_predicates,good_predicates,reworked_part
0,0,The indirect relationship between COVID-19 and...,ok,[],"[[COVID-19, inflammatory response], [impact, C...",None
1,1,The scientific abstracts provided offer insigh...,ok,[],"[[treatments, inflammatory response]]",None
2,2,This cytokine storm is a hallmark of severe CO...,ok,[],[],None
3,3,The abstracts discuss the role of cytokines in...,ok,[],"[[BBB, cytokines], [Tocilizumab, CNS, 1], [cyt...",None
4,4,The abstract discussing the passage of cytokin...,ok,[],"[[BBB, cytokines], [cytokines, brain function]]",None
5,5,It specifically mentions that the amount of bl...,ok,[],"[[morphine, brain function]]",None
6,6,"Given this context, the indirect relationship ...",ok,[],"[[COVID-19, inflammatory response]]",None
7,7,These cytokines can cross the BBB and potentia...,ok,[],"[[cytokines, CNS function]]",None
8,8,"Morphine, a compound known to cross the BBB an...",REWORK,"[[Morphine, CNS function, 2]]",[],None
9,9,"For example, morphine's known immunosuppressiv...",ok,[],"[[immunosuppressive effects, COVID-19], [COVID...",None


In [85]:
expl_parts_enum_dict = dict(enumerate(evaluation_result))
rework_sentences = []

for key, value in expl_parts_enum_dict.items():
    if value['status'] == 'REWORK':  
        rework_sentences.append(value['sent_text']) 
print(rework_sentences)

['Morphine, a compound known to cross the BBB and affect CNS function, could theoretically interact with or modulate the CNS effects of these cytokines. ']


In [86]:
rework_sent_to_drop_abst_3 = rework_sentences[0]
rework_sent_to_drop_abst_3

'Morphine, a compound known to cross the BBB and affect CNS function, could theoretically interact with or modulate the CNS effects of these cytokines. '

### Find closest abstract for Rework sentence

In [87]:
cur_top_n_context_abstr_drop_list_2 = find_top_documents(rework_sent_to_drop_abst_1,results_path1, top_k=1,return_scores=False)
cur_top_n_context_abstr_drop_list_2

['Single-Cell RNA Sequencing of Tocilizumab-Treated Peripheral Blood Mononuclear Cells as an in vitro Model of Inflammation. COVID-19 has posed a significant threat to global health. Early data has revealed that IL-6, a key regulatory cytokine, plays an important role in the cytokine storm of COVID-19. Multiple trials are therefore looking at the effects of Tocilizumab, an IL-6 receptor antibody that inhibits IL-6 activity, on treatment of COVID-19, with promising findings. As part of a clinical trial looking at the effects of Tocilizumab treatment on kidney transplant recipients with subclinical rejection, we performed single-cell RNA sequencing of comparing stimulated PBMCs before and after Tocilizumab treatment. We leveraged this data to create an in vitro cytokine storm model, to better understand the effects of Tocilizumab in the presence of inflammation. Tocilizumab-treated cells had reduced expression of inflammatory-mediated genes and biologic pathways, particularly amongst mon

In [88]:
len(results_path1)

5

In [91]:
results_path1.pop(2)
results_path1

['Early Viral Clearance and Antibody Kinetics of COVID-19 Among Asymptomatic Carriers. Asymptomatic carriers contribute to the spread of Coronavirus Disease 2019 (COVID-19), but their clinical characteristics, viral kinetics, and antibody responses remain unclear. A total of 56 COVID-19 patients without symptoms at admission and 19 age-matched symptomatic patients were enrolled. RNA of SARS-CoV-2 was tested using transcriptase quantitative PCR, and the total antibodies (Ab), IgG, IgA, and IgM against the SARS-CoV-2 were tested using Chemiluminescence Microparticle Immuno Assay. Among 56 patients without symptoms at admission, 33 cases displayed symptoms and 23 remained asymptomatic throughout the follow-up period. 43.8% of the asymptomatic carriers were children and none of the asymptomatic cases had recognizable changes in C-reactive protein or interleukin-6, except one 64-year-old patient. The initial threshold cycle value of nasopharyngeal SARS-CoV-2 in asymptomatic carriers was sim

### Iteration 3

In [92]:
llm_fix_prompt_nosummary= 'How would you describe plausible explanation of potential connection between Covid Virus Disease and morphine given the following scientific abstracts as context?'
llm_fix_prompt_str = '\n\n'.join(results_path1)
llm_fix_prompt_combo = llm_fix_prompt_nosummary + '\n' + llm_fix_prompt_str
llm_fix_prompt_combo

"How would you describe plausible explanation of potential connection between Covid Virus Disease and morphine given the following scientific abstracts as context?\nEarly Viral Clearance and Antibody Kinetics of COVID-19 Among Asymptomatic Carriers. Asymptomatic carriers contribute to the spread of Coronavirus Disease 2019 (COVID-19), but their clinical characteristics, viral kinetics, and antibody responses remain unclear. A total of 56 COVID-19 patients without symptoms at admission and 19 age-matched symptomatic patients were enrolled. RNA of SARS-CoV-2 was tested using transcriptase quantitative PCR, and the total antibodies (Ab), IgG, IgA, and IgM against the SARS-CoV-2 were tested using Chemiluminescence Microparticle Immuno Assay. Among 56 patients without symptoms at admission, 33 cases displayed symptoms and 23 remained asymptomatic throughout the follow-up period. 43.8% of the asymptomatic carriers were children and none of the asymptomatic cases had recognizable changes in C

In [93]:
%%time
llm_resp_reworked_2 = oai_get_response(llm_fix_prompt_combo,0)
llm_resp_reworked_2

CPU times: user 19.4 ms, sys: 8.11 ms, total: 27.5 ms
Wall time: 30 s


"Given the scientific abstracts provided, a plausible explanation of the potential connection between COVID-19 and morphine could be hypothesized through the lens of inflammation and the central nervous system's (CNS) response to infection and pain management.\n\n1. **COVID-19 and Inflammatory Response**: The abstracts highlight the significant role of inflammatory markers, such as interleukin-6 (IL-6) and tumor necrosis factor-alpha (TNF-alpha), in the pathophysiology of COVID-19. Elevated levels of these cytokines are associated with severe disease outcomes and are indicative of the cytokine storm syndrome observed in critical COVID-19 cases.\n\n2. **Morphine and Inflammation**: Morphine is known to have immunomodulatory effects, potentially influencing the inflammatory response. While not directly mentioned in the abstracts, morphine's ability to modulate the immune response could theoretically impact the progression of COVID-19 by either exacerbating or alleviating the cytokine sto

In [94]:
evaluation_result = evaluate_llm_resp(llm_resp_reworked_2)

This is original generated LLM res: Given the scientific abstracts provided, a plausible explanation of the potential connection between COVID-19 and morphine could be hypothesized through the lens of inflammation and the central nervous system's (CNS) response to infection and pain management.

1. **COVID-19 and Inflammatory Response**: The abstracts highlight the significant role of inflammatory markers, such as interleukin-6 (IL-6) and tumor necrosis factor-alpha (TNF-alpha), in the pathophysiology of COVID-19. Elevated levels of these cytokines are associated with severe disease outcomes and are indicative of the cytokine storm syndrome observed in critical COVID-19 cases.

2. **Morphine and Inflammation**: Morphine is known to have immunomodulatory effects, potentially influencing the inflammatory response. While not directly mentioned in the abstracts, morphine's ability to modulate the immune response could theoretically impact the progression of COVID-19 by either exacerbating 

In [95]:
expl_parts_df = pd.DataFrame(evaluation_result).reset_index()
expl_parts_df['reworked_part'] = None
expl_parts_df

,index,sent_text,status,bad_predicates,good_predicates,reworked_part
0,0,"Given the scientific abstracts provided, a pla...",ok,[],[],None
1,1,COVID-19 and Inflammatory Response: The abstra...,ok,[],[],None
2,2,Elevated levels of these cytokines are associa...,ok,[],[],None
3,3,Morphine and Inflammation: Morphine is known t...,ok,[],"[[Morphine, immunomodulatory]]",None
4,4,"While not directly mentioned in the abstracts,...",ok,[],[],None
5,5,"However, the exact effect would depend on the ...",ok,[],[],None
6,6,Cytokines and the Blood-Brain Barrier (BBB): T...,ok,[],"[[BBB, cytokines]]",None
7,7,Morphine's known ability to cross the BBB and ...,ok,[],[],None
8,8,Potential Therapeutic Implications: Considerin...,REWORK,"[[TNF-alpha inhibitor, postoperative pain, 2]]","[[TNF-alpha inhibitor, inflammation]]",None
9,9,Morphine could theoretically influence the CNS...,ok,[],"[[inflammation, COVID-19], [inflammation, cyto...",None


In [96]:
expl_parts_enum_dict = dict(enumerate(evaluation_result))
rework_sentences = []

for key, value in expl_parts_enum_dict.items():
    if value['status'] == 'REWORK':  
        rework_sentences.append(value['sent_text']) 
print(rework_sentences)

['Potential Therapeutic Implications: Considering the abstract discussing the effect of pentoxifylline (a TNF-alpha inhibitor) on reducing postoperative pain and inflammation, one could speculate about the potential use of morphine not only for its analgesic properties but also for its potential effects on the inflammatory response in COVID-19. ']


In [97]:
rework_sent_to_drop_abst_6= rework_sentences[0]
rework_sent_to_drop_abst_6

'Potential Therapeutic Implications: Considering the abstract discussing the effect of pentoxifylline (a TNF-alpha inhibitor) on reducing postoperative pain and inflammation, one could speculate about the potential use of morphine not only for its analgesic properties but also for its potential effects on the inflammatory response in COVID-19. '

In [98]:
len(results_path1)

4

In [99]:
cur_top_n_context_abstr_drop_list_2 = find_top_documents(rework_sent_to_drop_abst_6,results_path1, top_k=1,return_scores=False)
cur_top_n_context_abstr_drop_list_2

['Effect of intravenous pentoxifylline in inflammatory response in patients undergoing nephrolithotomy. To determine the potential efficacy of intravenous (IV) infusion of pentoxifylline (PTX) before nephrolithotomy on attenuating plasma level of the tumor necrosis factor (TNF)-alpha and interleukin (IL)-1, and to investigate whether it prevents postoperative pain. In a randomized, double-blind, placebo-controlled study, 32 patients (American Society of Anesthesiologists physical status 1 and 2) who were undergoing general anesthesia for nephrolithotomy were randomized to receive intravenous PTX (500 mg in 500 mL saline for 2 hours followed by 700 mg in 1000 mL saline for 6 hours), or placebo (1500 mL saline) before induction of the anesthesia. Two venous blood samples were obtained 10 minutes before PTX or placebo infusion and after surgery at 24 hours for laboratory examination. After surgery, the amount of narcotics consumption and intensity of pain (Visual Analog Scale and Verbal R

In [101]:
results_path1.pop(3)
results_path1

['Early Viral Clearance and Antibody Kinetics of COVID-19 Among Asymptomatic Carriers. Asymptomatic carriers contribute to the spread of Coronavirus Disease 2019 (COVID-19), but their clinical characteristics, viral kinetics, and antibody responses remain unclear. A total of 56 COVID-19 patients without symptoms at admission and 19 age-matched symptomatic patients were enrolled. RNA of SARS-CoV-2 was tested using transcriptase quantitative PCR, and the total antibodies (Ab), IgG, IgA, and IgM against the SARS-CoV-2 were tested using Chemiluminescence Microparticle Immuno Assay. Among 56 patients without symptoms at admission, 33 cases displayed symptoms and 23 remained asymptomatic throughout the follow-up period. 43.8% of the asymptomatic carriers were children and none of the asymptomatic cases had recognizable changes in C-reactive protein or interleukin-6, except one 64-year-old patient. The initial threshold cycle value of nasopharyngeal SARS-CoV-2 in asymptomatic carriers was sim

In [102]:
len(results_path1)

3

### Iteration 4

In [103]:
llm_fix_prompt_nosummary= 'How would you describe plausible explanation of potential connection between Covid Virus Disease and morphine given the following scientific abstracts as context?'
llm_fix_prompt_str = '\n\n'.join(results_path1)
llm_fix_prompt_combo = llm_fix_prompt_nosummary + '\n' + llm_fix_prompt_str
llm_fix_prompt_combo

"How would you describe plausible explanation of potential connection between Covid Virus Disease and morphine given the following scientific abstracts as context?\nEarly Viral Clearance and Antibody Kinetics of COVID-19 Among Asymptomatic Carriers. Asymptomatic carriers contribute to the spread of Coronavirus Disease 2019 (COVID-19), but their clinical characteristics, viral kinetics, and antibody responses remain unclear. A total of 56 COVID-19 patients without symptoms at admission and 19 age-matched symptomatic patients were enrolled. RNA of SARS-CoV-2 was tested using transcriptase quantitative PCR, and the total antibodies (Ab), IgG, IgA, and IgM against the SARS-CoV-2 were tested using Chemiluminescence Microparticle Immuno Assay. Among 56 patients without symptoms at admission, 33 cases displayed symptoms and 23 remained asymptomatic throughout the follow-up period. 43.8% of the asymptomatic carriers were children and none of the asymptomatic cases had recognizable changes in C

In [104]:
%%time
llm_resp_reworked_6 = oai_get_response(llm_fix_prompt_combo,0)
llm_resp_reworked_6

CPU times: user 16 ms, sys: 9.27 ms, total: 25.2 ms
Wall time: 26.6 s


"Given the context provided by the scientific abstracts, a plausible explanation for a potential connection between COVID-19 and morphine could be hypothesized around the management of symptoms and the modulation of the immune response in COVID-19 patients. However, it's important to note that none of the abstracts directly address the use of morphine in COVID-19 patients. Instead, they focus on the characteristics of asymptomatic carriers, predictive models for COVID-19 pneumonia severity, and the effects of pentoxifylline (PTX) on inflammatory responses in a surgical context. To draw a connection, we must extrapolate from the information provided and known pharmacological principles.\n\n1. **Management of Symptoms in COVID-19**: The abstracts do not directly mention morphine, but morphine is a potent opioid analgesic commonly used to manage severe pain. In the context of COVID-19, patients with severe pneumonia and those requiring high-flow-oxygen-support might experience significant

In [105]:
%%time
evaluation_result = evaluate_llm_resp(llm_resp_reworked_6)

This is original generated LLM res: Given the context provided by the scientific abstracts, a plausible explanation for a potential connection between COVID-19 and morphine could be hypothesized around the management of symptoms and the modulation of the immune response in COVID-19 patients. However, it's important to note that none of the abstracts directly address the use of morphine in COVID-19 patients. Instead, they focus on the characteristics of asymptomatic carriers, predictive models for COVID-19 pneumonia severity, and the effects of pentoxifylline (PTX) on inflammatory responses in a surgical context. To draw a connection, we must extrapolate from the information provided and known pharmacological principles.

1. **Management of Symptoms in COVID-19**: The abstracts do not directly mention morphine, but morphine is a potent opioid analgesic commonly used to manage severe pain. In the context of COVID-19, patients with severe pneumonia and those requiring high-flow-oxygen-sup

In [106]:
expl_parts_df = pd.DataFrame(evaluation_result).reset_index()
expl_parts_df['reworked_part'] = None
expl_parts_df

,index,sent_text,status,bad_predicates,good_predicates,reworked_part
0,0,Given the context provided by the scientific a...,ok,[],"[[COVID-19, patients], [morphine, symptoms], [...",None
1,1,"However, it's important to note that none of t...",ok,[],"[[morphine, COVID-19], [COVID-19, patients], [...",None
2,2,"Instead, they focus on the characteristics of ...",ok,[],"[[pentoxifylline, inflammatory responses]]",None
3,3,"To draw a connection, we must extrapolate from...",ok,[],[],None
4,4,Management of Symptoms in COVID-19: The abstra...,ok,[],"[[Symptoms, COVID-19], [opioid analgesic, seve...",None
5,5,"In the context of COVID-19, patients with seve...",ok,[],"[[pain, patients], [discomfort, patients], [pn...",None
6,6,The predictive models for COVID-19 pneumonia s...,ok,[],[],None
7,7,Modulation of the Immune Response: The third a...,ok,[],"[[nephrolithotomy, inflammatory responses], [n...",None
8,8,"While PTX is not morphine, this finding is rel...",ok,[],"[[drugs, immune response]]",None
9,9,Morphine has been shown in various studies to ...,ok,[],"[[Morphine, immunomodulatory]]",None


No Rework Sentence find out so we end up path here.